In [ ]:
import csv, os, shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

csvfile = open('M:\masters\semester 4\web scraping\output files\TopicStats\Syria.csv', newline='') 
word2vecFolderDirectory = "M:\masters\semester 4\ProjectData\SyriaSpeechesRdf2vec"
spamreader = csv.reader(csvfile, delimiter=',')

MPNames = [] #where the MP Names are saved (actual list from syria speeches folder)
MPParty =[] #where the MP party will be saved (should have same index as MPNames)
CSVStatsMpNames = [] #where the MP Names from the CSV Statistic file are saved
CSVStatsMpParty = [] #where the MP Party from the SCV Statistics file are saved
word2vecFileTextList = [] #where the TagMe TextFiles are Saved

#function that returns the MPs party name and it takes as input the list with all mp names, list with their corresponding parties
# as well as the name of the MP we are looking for.
def getParty(CSVStatsMpNames, CSVStatsMpParty, name):
    for mpnames in CSVStatsMpNames:
        index = CSVStatsMpNames.index(mpnames)
        if(name.strip() in mpnames.strip()):
            mpparty = CSVStatsMpParty[index]
            return mpparty

for file in os.listdir(word2vecFolderDirectory): #save the text from the syria files
    filedirectorylink = os.path.join(word2vecFolderDirectory,file)
    fileText = open(filedirectorylink, encoding="utf8")
    lista = [float(x) for x in fileText.read().split(" ") if len(x)>2] #this transforms the file and the content from str to float
    word2vecFileTextList.append(lista)
        
for fileName in os.listdir(word2vecFolderDirectory): #save the names to correspond the texts
    NameList = fileName.strip().split("_")
    MpName = ""
    if(len(NameList) >= 2):
        MpName = NameList[0] +" "+ NameList[1]
    else:
        MpName = NameList[0]
    MPNames.append(MpName)

for row in spamreader: #saving mpnames and parties from the statistics file
    mpname = row[0].strip()
    mpparty = row[1].strip()
    if(mpname not in CSVStatsMpNames):
        CSVStatsMpNames.append(mpname)
        CSVStatsMpParty.append(mpparty)
print(len(CSVStatsMpNames))
print(len(CSVStatsMpParty))

for MP in MPNames: #matchin MPnames from the folder with their parties
    index = MPNames.index(MP)
    mpparty = getParty(CSVStatsMpNames,CSVStatsMpParty, MP.strip())
    MPParty.insert(index, mpparty)
    
#tfidf = TfidfVectorizer().fit_transform(word2vecFileTextList) no need for tfidf because they are already vectors
#cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
#it gets the matrix as an input example, "[len(TagMeFileTextList)-1:len(TagMeFileTextList)]" or "[0:1]"
counter1 = 0
counter2 = 1
for file in word2vecFileTextList:
    cosine_similarities = cosine_similarity(word2vecFileTextList[counter1:counter2], word2vecFileTextList).flatten() 
    print(cosine_similarities)
    related_docs_indices = cosine_similarities.argsort()[:-6:-1] #first 5 related elements
    print(related_docs_indices)
    RelatedDocsParties = []
    RelatedDocMps =[]
    for doc in related_docs_indices:
        docparty = MPParty[doc]
        docMp = MPNames[doc]
        RelatedDocMps.append(docMp)
        RelatedDocsParties.append(docparty)
    
    counter1 = counter1 + 1 #counters are used to loop each file to compare it with all the others
    counter2 = counter2 + 1 #counters are used to loop each file to compare it with all the others
    
    print("related doc mps: ", RelatedDocMps)    
    print("related doc parties: ", RelatedDocsParties)
    print("related doc indices :" ,related_docs_indices)
    print(len(related_docs_indices))
    print(cosine_similarities[related_docs_indices])

